In [ ]:
import wandb
import numpy as np
import pandas as pd

In [ ]:
api = wandb.Api(timeout=120)
runs = api.runs("wilrop/MORL-Baselines")

In [ ]:
run_hists = {
    'deep-sea-treasure-concave-v0': {'PCN': [], 'GPI-LS': [], 'Envelope': []},
    'minecart-v0': {'PCN': [], 'GPI-LS': [], 'Envelope': []},
    'mo-reacher-v4': {'PCN': [], 'GPI-LS': [], 'Envelope': []}
}

for run in runs:
    env_id = run.config['env_id']
    alg = run.config['algo']
    run_hists[env_id][alg].append(run.history(keys=['eval/hypervolume', 'global_step']))
    print(f'Added run to {env_id} - {alg}')

In [ ]:
def fill_hvs(hists, num_samples):    
    min_logs = min(len(hist['eval/hypervolume']) for hist in hists)
    indices = np.linspace(0, min_logs - 1, num_samples).astype(int)
    all_global_steps = []
    
    for hist in hists:
        steps = hist['global_step'].to_numpy()
        new_steps = steps[indices]
        all_global_steps.append(new_steps)
        
    global_steps = np.sort(np.unique(np.concatenate(all_global_steps)))
    all_hypervolumes = []
    
    for hist in hists:
        new_hv = []
        hypervolumes = hist['eval/hypervolume'].to_numpy()
        hv_steps = hist['global_step'].to_numpy()
        for step in global_steps:
            idx = np.argmin(np.abs(hv_steps - step))
            new_hv.append(hypervolumes[idx])
        new_hv = np.insert(new_hv, 0, 0)
        all_hypervolumes.append(new_hv.tolist())
    
    global_steps = np.insert(global_steps, 0, 0)
    return global_steps, all_hypervolumes

In [ ]:
num_samples = 300
num_seeds = 5
for env_id, alg_hists in run_hists.items():
    for alg, hists in alg_hists.items():
        if not hists:
            continue
        global_steps, hvs = fill_hvs(hists, num_samples)
        hv_dict = {alg: [], 'Step': [], 'Seed': []}
            
        for seed, hv in enumerate(hvs):
            hv_dict[alg].extend(hv)
            hv_dict['Step'].extend(global_steps)
            hv_dict['Seed'].extend([seed] * len(global_steps))

        hv_df = pd.DataFrame.from_dict(hv_dict)
        print(f"Saving data for {alg}")
        hv_df.to_csv(f'data/{alg}_{env_id}_hv.csv', index=False)